In [2]:
pip install torch torchvision opencv-python numpy


Note: you may need to restart the kernel to use updated packages.


In [9]:
import cv2
import os
from datetime import datetime
from ultralytics import YOLO

# ========= CONFIGURATION =========
IP_CAM_URL = 'http://192.168.129.195:4747/video'  
MODEL_PATH_1 = 'best _Garbage.pt'  # Garbage detection
MODEL_PATH_2 = 'best_Spills.pt'  # Spill detection
TARGET_CLASSES = ['garbage', 'spill']  # Trigger snapshot if detected
SNAPSHOT_DIR = 'detections'

# ========= SETUP =========
os.makedirs(SNAPSHOT_DIR, exist_ok=True)

model1 = YOLO(MODEL_PATH_1)
model2 = YOLO(MODEL_PATH_2)

cap = cv2.VideoCapture(IP_CAM_URL)

print("[INFO] Starting YOLOv8 Detection... Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read video frame.")
        break

    # Run inference with both modelsq
    results1 = model1(frame, verbose=False)
    results2 = model2(frame, verbose=False)

    detected = False

    # Combine both results
    for results in [results1, results2]:
        for r in results:
            boxes = r.boxes
            for box in boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                label = r.names[cls_id]
                x1, y1, x2, y2 = xyxy

                # Draw box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                # Snapshot trigger
                if label.lower() in TARGET_CLASSES and conf > 0.5:
                    detected = True

    # Save snapshot if target detected
    if detected:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = os.path.join(SNAPSHOT_DIR, f'detection_{timestamp}.jpg')
        cv2.imwrite(filename, frame)
        print(f"[📸] Snapshot saved: {filename}")

    # Show detection window
    cv2.imshow("YOLOv8 Live Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[INFO] Starting YOLOv8 Detection... Press 'q' to quit.
[📸] Snapshot saved: detections\detection_20250417_012749.jpg
[📸] Snapshot saved: detections\detection_20250417_012815.jpg
[📸] Snapshot saved: detections\detection_20250417_012815.jpg
[📸] Snapshot saved: detections\detection_20250417_012815.jpg
[📸] Snapshot saved: detections\detection_20250417_012815.jpg
[📸] Snapshot saved: detections\detection_20250417_012815.jpg
[📸] Snapshot saved: detections\detection_20250417_012816.jpg
[📸] Snapshot saved: detections\detection_20250417_012816.jpg
[📸] Snapshot saved: detections\detection_20250417_012816.jpg
[📸] Snapshot saved: detections\detection_20250417_012816.jpg
[📸] Snapshot saved: detections\detection_20250417_012817.jpg
[📸] Snapshot saved: detections\detection_20250417_012817.jpg
[📸] Snapshot saved: detections\detection_20250417_012817.jpg
[📸] Snapshot saved: detections\detection_20250417_012817.jpg
[📸] Snapshot saved: detections\detection_20250417_012817.jpg
[📸] Snapshot saved: detections

In [31]:
import cv2
import os
import time
from datetime import datetime
from ultralytics import YOLO

# ========= CONFIGURATION =========
IP_CAM_URL = 'http://192.168.129.195:4747/video'  # Camera URL
MODEL_PATH_1 = 'best_Garbage.pt'  # Garbage detection model
MODEL_PATH_2 = 'best_Spills.pt'   # Spill detection model
TARGET_CLASSES = ['garbage', 'spill']  # Targeted classes for snapshot saving
SNAPSHOT_DIR = r'C:\Users\mahesh\Documents\result'  # Snapshot base directory

# ========= SETUP =========
try:
    os.makedirs(SNAPSHOT_DIR, exist_ok=True)
    print(f"[✔] Base directory ready: {SNAPSHOT_DIR}")
except Exception as e:
    print(f"❌ Error creating base directory: {e}")
    SNAPSHOT_DIR = os.path.join(os.getcwd(), 'result')  # Fallback directory
    os.makedirs(SNAPSHOT_DIR, exist_ok=True)
    print(f"[⚠] Using fallback path: {SNAPSHOT_DIR}")

# Load models
model1 = YOLO(MODEL_PATH_1)  # Garbage detection
model2 = YOLO(MODEL_PATH_2)  # Spill detection

# Setup video capture
cap = cv2.VideoCapture(IP_CAM_URL)

print("[INFO] Starting YOLOv8 Detection... Press 'q' to quit.")

# Dictionary to track last snapshot time for each label, initialize with time.time()
snapshot_taken = {label: 0 for label in TARGET_CLASSES}

# Function to check if 5 minutes have passed
def can_take_snapshot(label):
    current_time = time.time()
    last_snapshot_time = snapshot_taken.get(label, 0)

    # Check if 5 minutes (300 seconds) have passed since last snapshot
    return current_time - last_snapshot_time >= 300

while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to read video frame.")
        break

    # Run inference with both models
    results1 = model1(frame, verbose=False)
    results2 = model2(frame, verbose=False)

    # Combine results from both models
    for results in [results1, results2]:
        for r in results:
            boxes = r.boxes
            for box in boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                xyxy = box.xyxy[0].cpu().numpy().astype(int)
                label = r.names[cls_id].strip().lower()
                x1, y1, x2, y2 = xyxy

                # Draw detection box and label
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

                # Save snapshot if label matches target, confidence is high, and 5 minutes have passed
                if label in TARGET_CLASSES and conf > 0.5 and can_take_snapshot(label):
                    label_folder = os.path.join(SNAPSHOT_DIR, label)

                    try:
                        # Ensure directory exists
                        if not os.path.exists(label_folder):
                            os.makedirs(label_folder, exist_ok=True)
                            print(f"[INFO] Created folder: {label_folder}")

                        # Create timestamped filename
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        filename = os.path.join(label_folder, f'{label}_{timestamp}.jpg')

                        # Save the image
                        if frame is not None:
                            saved = cv2.imwrite(filename, frame)
                            if saved:
                                print(f"[📸] Snapshot saved: {filename}")
                                snapshot_taken[label] = time.time()  # Update last snapshot time
                                print(f"[INFO] Camera location: {IP_CAM_URL}")
                            else:
                                raise Exception("cv2.imwrite() returned False.")
                        else:
                            raise Exception("Frame is None.")
                    except Exception as e:
                        print(f"❌ Failed to save snapshot for {label} at {label_folder}")
                        print(f"    Error: {e}")

    # Display the frame with detections
    cv2.imshow("YOLOv8 Live Detection", frame)
    
    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


[✔] Base directory ready: C:\Users\mahesh\Documents\result
[INFO] Starting YOLOv8 Detection... Press 'q' to quit.
❌ Failed to save snapshot for garbage at C:\Users\mahesh\Documents\result\garbage
    Error: [WinError 2] The system cannot find the file specified: 'C:\\Users\\mahesh\\Documents\\result\\garbage'
❌ Failed to save snapshot for spill at C:\Users\mahesh\Documents\result\spill
    Error: [WinError 2] The system cannot find the file specified: 'C:\\Users\\mahesh\\Documents\\result\\spill'
❌ Failed to save snapshot for garbage at C:\Users\mahesh\Documents\result\garbage
    Error: [WinError 2] The system cannot find the file specified: 'C:\\Users\\mahesh\\Documents\\result\\garbage'
❌ Failed to save snapshot for garbage at C:\Users\mahesh\Documents\result\garbage
    Error: [WinError 2] The system cannot find the file specified: 'C:\\Users\\mahesh\\Documents\\result\\garbage'
❌ Failed to save snapshot for spill at C:\Users\mahesh\Documents\result\spill
    Error: [WinError 2] Th